In [ ]:
# Importação das bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import control.matlab as ctrl

In [ ]:
# Definindo a função de transferência do sistema
# Numerador: 0.25s + 0.10875
num = [0.25, 0.10875]

# Denominador: s^4 + 3.456s^3 + 3.45688s^2 + 0.719297s + 0.041574
den = [1, 3.456, 3.45688, 0.719297, 0.041574]

# Criando a função de transferência
sys = ctrl.TransferFunction(num, den)

# Exibindo a função de transferência
print("Função de Transferência do Sistema:")
print(sys)

In [ ]:
# Calculando os polos do sistema
polos = np.roots(den)
print("Polos do sistema:")
for i, polo in enumerate(polos):
    print(f"Polo {i+1}: {polo:.6f}")

# Calculando os zeros do sistema
zeros = np.roots(num)
print("\nZeros do sistema:")
for i, zero in enumerate(zeros):
    print(f"Zero {i+1}: {zero:.6f}")

In [ ]:
# Plotando o lugar das raízes - Método correto
plt.figure(figsize=(10, 8))

# Usando a função rlocus corretamente
rlocus_data = ctrl.rlocus(sys, grid=True)
plt.title('Lugar das Raízes - Sistema Completo (4ª ordem)')
plt.xlabel('Parte Real')
plt.ylabel('Parte Imaginária')

# Adicionando informações sobre polos e zeros
plt.scatter(np.real(polos), np.imag(polos), color='red', marker='x', s=100, label='Polos', linewidth=2)
plt.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', s=100, label='Zeros', facecolors='none', edgecolors='b', linewidth=2)
plt.legend()

# Adicionando linhas de grade
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análise da estabilidade do sistema em malha aberta
print("Análise de Estabilidade em Malha Aberta:")
print("=" * 50)

# Verificando se todos os polos têm parte real negativa
sistema_estavel = all(np.real(polos) < 0)
print(f"Sistema é estável em malha aberta: {sistema_estavel}")

# Calculando margem de ganho e fase se possível
try:
    gm, pm, wg, wp = ctrl.margin(sys)
    print(f"Margem de Ganho: {gm:.4f}")
    print(f"Margem de Fase: {pm:.4f}°")
except:
    print("Não foi possível calcular margens de estabilidade")

In [ ]:
# Plotando resposta ao degrau para análise do comportamento dinâmico
t = np.linspace(0, 50, 1000)
t, y = ctrl.step(sys, t)

plt.figure(figsize=(10, 6))
plt.plot(t, y)
plt.title('Resposta ao Degrau - Sistema em Malha Aberta')
plt.xlabel('Tempo (s)')
plt.ylabel('Ângulo de Arfagem θ')
plt.grid(True)
plt.show()

In [ ]:
# Análise mais detalhada do lugar das raízes
# Vamos calcular pontos específicos do lugar das raízes para diferentes ganhos
k_values = np.logspace(-3, 2, 50)  # Valores de ganho de 0.001 a 100

plt.figure(figsize=(12, 10))

# Criar uma lista para armazenar os pontos do lugar das raízes
all_real = []
all_imag = []
all_k_log = []

# Coletar dados do lugar das raízes
for k in k_values:
    # Sistema em malha fechada com ganho k
    sys_mf = ctrl.feedback(k * sys, 1)
    polos_mf = ctrl.pole(sys_mf)
    
    # Armazenar os dados
    for polo in polos_mf:
        all_real.append(np.real(polo))
        all_imag.append(np.imag(polo))
        all_k_log.append(np.log10(k))

# Plotar todos os pontos de uma vez com cores baseadas no ganho
scatter = plt.scatter(all_real, all_imag, c=all_k_log, cmap='viridis', 
                     marker='.', s=50, alpha=0.7)

# Adicionando polos e zeros originais
plt.scatter(np.real(polos), np.imag(polos), color='red', marker='x', s=100, 
           label='Polos (K=0)', linewidth=2)
plt.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', s=100, 
           label='Zeros', facecolors='none', edgecolors='b', linewidth=2)

# Adicionando linhas de amortecimento constante (ζ = 0.5)
theta = np.linspace(0, np.pi, 100)
zeta = 0.5
x_zeta = -np.cos(theta) * 5
y_zeta = np.sin(theta) * 5
plt.plot(x_zeta, y_zeta, 'r--', alpha=0.5, label='ζ = 0.5')
plt.plot(x_zeta, -y_zeta, 'r--', alpha=0.5)

plt.title('Lugar das Raízes Detalhado - Sistema Completo')
plt.xlabel('Parte Real')
plt.ylabel('Parte Imaginária')
plt.grid(True)
plt.axis([-4, 1, -2, 2])
plt.legend()

# Adicionando barra de cores para indicar o ganho
cbar = plt.colorbar(scatter)
cbar.set_label('log10(K)')

plt.tight_layout()
plt.show()

In [ ]:
# Versão alternativa mais simples do lugar das raízes detalhado
plt.figure(figsize=(12, 10))

# Plotando o lugar das raízes com cores baseadas no ganho
for k in k_values:
    # Sistema em malha fechada com ganho k
    sys_mf = ctrl.feedback(k * sys, 1)
    polos_mf = ctrl.pole(sys_mf)
    
    # Escolher cor baseada no ganho (escala logarítmica)
    color_intensity = (np.log10(k) + 3) / 5  # Normalizar entre 0 e 1
    color = plt.cm.viridis(color_intensity)
    
    plt.scatter(np.real(polos_mf), np.imag(polos_mf), 
               color=color, marker='.', s=50, alpha=0.7)

# Adicionando polos e zeros originais
plt.scatter(np.real(polos), np.imag(polos), color='red', marker='x', s=100, 
           label='Polos (K=0)', linewidth=2)
plt.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', s=100, 
           label='Zeros', facecolors='none', edgecolors='b', linewidth=2)

# Adicionando linhas de amortecimento constante
theta = np.linspace(0, np.pi, 100)
zeta = 0.5
x_zeta = -np.cos(theta) * 5
y_zeta = np.sin(theta) * 5
plt.plot(x_zeta, y_zeta, 'r--', alpha=0.5, label='ζ = 0.5')
plt.plot(x_zeta, -y_zeta, 'r--', alpha=0.5)

plt.title('Lugar das Raízes Detalhado - Sistema Completo')
plt.xlabel('Parte Real')
plt.ylabel('Parte Imaginária')
plt.grid(True)
plt.axis([-4, 1, -2, 2])
plt.legend()

# Adicionar uma barra de cores simples manualmente
plt.text(1.2, 0.5, 'K aumenta →', rotation=90, transform=plt.gca().transAxes,
         verticalalignment='center', fontsize=12)
plt.text(1.25, 0.2, 'K baixo', transform=plt.gca().transAxes,
         verticalalignment='center', fontsize=10, color='darkblue')
plt.text(1.25, 0.8, 'K alto', transform=plt.gca().transAxes,
         verticalalignment='center', fontsize=10, color='yellow')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from control import tf, root_locus

# --- Definição da Função de Transferência ---
# Coeficientes do numerador: 0.25s + 0.10875
num = [0.25, 0.10875]

# Coeficientes do denominador: s^4 + 3.456s^3 + 3.45688s^2 + 0.719297s + 0.041574
den = [1, 3.456, 3.45688, 0.719297, 0.041574]

# Criação do objeto da Função de Transferência
g = tf(num, den)

# --- Análise Preliminar do Sistema ---
print("Função de Transferência da Planta G(s):")
print(g)

# Calcula e exibe os polos e zeros da planta em malha aberta
poles = np.roots(den)
zeros = np.roots(num)

print("\nPolos de malha aberta:")
print(poles)

print("\nZeros de malha aberta:")
print(zeros)


# --- Geração do Gráfico do Lugar das Raízes ---
# Define o tamanho da figura para melhor visualização
plt.figure(figsize=(12, 8))

# Plota o lugar das raízes com grid
# A função retorna as localizações das raízes (rlist) e os ganhos (klist)
rlist, klist = root_locus(g, plot=True, grid=True)

# Adiciona um título e rótulos aos eixos para clareza
plt.title('Lugar das Raízes para a Planta do Veículo Submersível', fontsize=16)
plt.xlabel('Eixo Real (σ)', fontsize=12)
plt.ylabel('Eixo Imaginário (jω)', fontsize=12)

# Salva a figura em um arquivo para uso posterior
plt.savefig('lugar_das_raizes_veiculo.png')

# Exibe o gráfico final
plt.show()# 📊 Item (a): Resposta ao Degrau Unitário - VERSÃO REFATORADA
# 🎯 Análise criteriosa considerando "pegadinhas" do professor

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import control.matlab as ctrl
from scipy import linalg
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("ITEM (a) - RESPOSTA AO DEGRAU UNITÁRIO - ANÁLISE CRITERIOSA")
print("="*80)

# =============================================================================
# 1. VERIFICAÇÃO RIGOROSA DOS DADOS DO PROBLEMA
# =============================================================================

print("\n🔍 1. VERIFICAÇÃO DOS DADOS DO PROBLEMA")

# Dados fornecidos - VERIFICAR SINAIS E CONSTANTES!
num_nominal = [0.25, 0.10875]      # 0.25s + 0.10875
den_nominal = [1, 3.456, 3.45688, 0.719297, 0.041574]

# ⚠️ PEGADINHA POSSÍVEL: Verificar se os sinais estão corretos
print("📋 Dados fornecidos:")
print(f"Numerador: {num_nominal}")
print(f"Denominador: {den_nominal}")

# Verificar consistência dos coeficientes
def verificar_consistencia(num, den):
    """Verifica consistência matemática dos coeficientes"""
    issues = []
    
    # Verificar se o sistema é próprio (grau num <= grau den)
    if len(num) > len(den):
        issues.append("❌ Sistema impróprio (grau num > grau den)")
    
    # Verificar valores muito pequenos que podem ser ruído
    if any(abs(coef) < 1e-10 for coef in num + den):
        issues.append("⚠️  Coeficientes muito pequenos (possível ruído)")
    
    # Verificar se o denominador é mônico
    if den[0] != 1:
        issues.append("⚠️  Denominador não é mônico")
    
    return issues

issues = verificar_consistencia(num_nominal, den_nominal)
if issues:
    print("📢 AVISOS DE CONSISTÊNCIA:")
    for issue in issues:
        print(f"   {issue}")

# =============================================================================
# 2. ANÁLISE DE ESTABILIDADE E SINGULARIDADES
# =============================================================================

print("\n📊 2. ANÁLISE DE ESTABILIDADE E SINGULARIDADES")

# Criar sistema com verificação de erro
try:
    sys = ctrl.TransferFunction(num_nominal, den_nominal)
    print("✅ Sistema criado com sucesso")
except Exception as e:
    print(f"❌ Erro ao criar sistema: {e}")
    # Tentar normalizar denominador
    den_normalized = [coef/den_nominal[0] for coef in den_nominal]
    sys = ctrl.TransferFunction(num_nominal, den_normalized)
    print("✅ Sistema criado com denominador normalizado")

# Calcular polos com verificação numérica
polos = ctrl.pole(sys)
print(f"📈 Número de polos: {len(polos)}")

# Verificar estabilidade rigorosa
def analisar_estabilidade(polos, tol=1e-10):
    """Análise rigorosa de estabilidade"""
    
    polos_reais = np.real(polos)
    polos_imag = np.imag(polos)
    
    # Polos no semiplano direito
    polos_instaveis = polos[polos_reais > tol]
    polos_marginais = polos[np.abs(polos_reais) <= tol]  # Polos no eixo imaginário
    
    print(f"🔍 Análise de estabilidade (tol = {tol}):")
    print(f"   • Polos estáveis: {np.sum(polos_reais < -tol)}")
    print(f"   • Polos marginalmente estáveis: {len(polos_marginais)}")
    print(f"   • Polos instáveis: {len(polos_instaveis)}")
    
    if len(polos_instaveis) > 0:
        print(f"   ❌ SISTEMA INSTÁVEL - Polos instáveis: {polos_instaveis}")
        return False
    elif len(polos_marginais) > 0:
        print(f"   ⚠️  SISTEMA MARGINALMENTE ESTÁVEL - Polos no eixo: {polos_marginais}")
        return False
    else:
        print("   ✅ SISTEMA ESTÁVEL")
        return True

estavel = analisar_estabilidade(polos)

# Análise detalhada dos polos
print("\n📋 Localização dos polos:")
for i, polo in enumerate(polos, 1):
    tipo = "REAL" if abs(polo.imag) < 1e-10 else "COMPLEXO"
    print(f"   Polo {i}: {polo.real:10.6f} {polo.imag:+.6f}j ({tipo})")

# =============================================================================
# 3. CÁLCULO DA RESPOSTA AO DEGRAU - MÉTODO ROBUSTO
# =============================================================================

print("\n🧮 3. CÁLCULO DA RESPOSTA AO DEGRAU")

# Método 1: Usando control.matlab (mais robusto)
try:
    t, y_step = ctrl.step(sys, T=np.linspace(0, 100, 5000))
    print("✅ Resposta ao degrau calculada (método control.matlab)")
except:
    # Método 2: Usando scipy (fallback)
    t, y_step = signal.step((num_nominal, den_nominal), T=np.linspace(0, 100, 5000))
    print("✅ Resposta ao degrau calculada (método scipy)")

# Verificar convergência
valor_final = y_step[-1] if len(y_step) > 0 else 0
valor_inicial = y_step[0] if len(y_step) > 0 else 0

print(f"📊 Características da resposta:")
print(f"   • Valor inicial: {valor_inicial:.6f}")
print(f"   • Valor final: {valor_final:.6f}")

# Verificar se a resposta converge
if len(y_step) > 10:
    ultimos_10 = y_step[-10:]
    variacao_final = np.std(ultimos_10) / abs(np.mean(ultimos_10)) if np.mean(ultimos_10) != 0 else 0
    print(f"   • Variação nos últimos 10 pontos: {variacao_final:.2e}")
    
    if variacao_final > 0.01:  # Mais de 1% de variação
        print("   ⚠️  Resposta pode não ter convergido completamente")
    else:
        print("   ✅ Resposta convergiu adequadamente")

# =============================================================================
# 4. EXPANSÃO EM FRAÇÕES PARCIAIS - MÉTODO PRECISO
# =============================================================================

print("\n🔢 4. EXPANSÃO EM FRAÇÕES PARCIAIS")

def expansao_fracoes_parciais_robusta(num, den):
    """Expansão em frações parciais com verificação de precisão"""
    
    # Para resposta ao degrau: θ(s) = G(s)/s
    # Adicionar polo em s=0 ao denominador
    den_expandido = np.convolve(den, [1, 0])
    
    print("📐 Expansão para θ(s) = G(s)/s")
    print(f"   Denominador expandido: {den_expandido}")
    
    # Calcular resíduos usando scipy (mais preciso)
    try:
        r, p, k = signal.residue(num, den_expandido)
        
        print("✅ Expansão calculada com sucesso")
        print(f"   Número de termos: {len(r)}")
        
        # Verificar precisão dos resíduos
        erro_reconstrucao = verificar_precisao_expansao(r, p, k, num, den_expandido)
        print(f"   Erro de reconstrução: {erro_reconstrucao:.2e}")
        
        if erro_reconstrucao > 1e-6:
            print("   ⚠️  Erro de reconstrução elevado")
        
        return r, p, k
        
    except Exception as e:
        print(f"❌ Erro na expansão: {e}")
        return None, None, None

def verificar_precisao_expansao(r, p, k, num_original, den_original):
    """Verifica a precisão da expansão reconstruindo a FT"""
    from scipy import polyval
    
    # Reconstruir a função a partir da expansão
    s_test = 1.0  # Ponto de teste
    valor_reconstruido = 0
    
    # Termos das frações parciais
    for i in range(len(r)):
        valor_reconstruido += r[i] / (s_test - p[i])
    
    # Termo direto (se existir)
    if len(k) > 0:
        valor_reconstruido += polyval(k, s_test)
    
    # Valor original
    valor_original = polyval(num_original, s_test) / polyval(den_original, s_test)
    
    return abs(valor_reconstruido - valor_original)

# Calcular expansão
r, p, k = expansao_fracoes_parciais_robusta(num_nominal, den_nominal)

if r is not None:
    print("\n📊 Coeficientes da expansão:")
    for i, (residuo, polo) in enumerate(zip(r, p), 1):
        if abs(polo) < 1e-10:
            print(f"   A{i} = {residuo:.6f} (polo em s = 0)")
        else:
            print(f"   A{i} = {residuo:.6f} (polo em s = {polo:.6f})")

# =============================================================================
# 5. TRANSFORMADA INVERSA DE LAPLACE - IMPLEMENTAÇÃO PRECISA
# =============================================================================

print("\n🔄 5. TRANSFORMADA INVERSA DE LAPLACE")

def calcular_resposta_temporal(r, p, k, t):
    """Calcula a resposta temporal a partir da expansão"""
    
    theta_t = np.zeros_like(t)
    
    # Adicionar contribuição de cada termo
    for i, (residuo, polo) in enumerate(zip(r, p)):
        if abs(polo) < 1e-10:  # Polo em s=0
            theta_t += residuo * np.ones_like(t)
        elif abs(polo.imag) < 1e-10:  # Polo real
            theta_t += residuo * np.exp(polo * t)
        else:  # Polo complexo
            # Para polos complexos conjugados, processar em pares
            if polo.imag > 0:  # Apenas processar uma vez
                # Encontrar o conjugado
                for j, (r_conj, p_conj) in enumerate(zip(r, p)):
                    if j > i and abs(p_conj - np.conj(polo)) < 1e-10:
                        # Combinar termos complexos conjugados
                        sigma = polo.real
                        omega = polo.imag
                        magnitude = 2 * abs(residuo)
                        fase = np.angle(residuo)
                        
                        theta_t += magnitude * np.exp(sigma * t) * np.cos(omega * t + fase)
                        break
    
    # Aplicar degrau unitário
    theta_t = theta_t * (t >= 0)
    
    return theta_t

if r is not None:
    # Calcular resposta temporal analítica
    theta_analitico = calcular_resposta_temporal(r, p, k, t)
    
    # Comparar com a resposta numérica
    erro_analitico_numerico = np.sqrt(np.mean((theta_analitico - y_step)**2))
    print(f"✅ Resposta temporal analítica calculada")
    print(f"   Erro RMS entre métodos analítico e numérico: {erro_analitico_numerico:.2e}")

# =============================================================================
# 6. ANÁLISE DE SENSIBILIDADE E ROBUSTEZ
# =============================================================================

print("\n🔎 6. ANÁLISE DE SENSIBILIDADE (PROFESSOR SACANA!)")

# Teste 1: Sensibilidade a pequenas variações nos parâmetros
print("📋 Teste de sensibilidade paramétrica:")
variacoes = [0.99, 1.0, 1.01]  # -1%, nominal, +1%

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, variacao in enumerate(variacoes):
    # Aplicar variação aos parâmetros
    num_var = [coef * variacao for coef in num_nominal]
    den_var = [coef * variacao for coef in den_nominal]
    
    try:
        sys_var = ctrl.TransferFunction(num_var, den_var)
        t_var, y_var = ctrl.step(sys_var, T=t)
        
        # Plot comparativo
        axes[0].plot(t_var, y_var, label=f'Variação {((variacao-1)*100):+.0f}%')
        axes[0].set_title('Sensibilidade Paramétrica')
        axes[0].set_xlabel('Tempo (s)')
        axes[0].set_ylabel('θ(t)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
    except Exception as e:
        print(f"   ⚠️  Erro com variação {variacao}: {e}")

# Teste 2: Análise de polos dominantes
print("\n📊 Análise de polos dominantes:")
polos_ordenados = sorted(polos, key=lambda x: abs(x.real))
print("   Polos ordenados por dominância:")
for i, polo in enumerate(polos_ordenados, 1):
    print(f"   {i}º: {polo.real:.6f} {polo.imag:+.6f}j")

# Identificar polos dominantes (mais próximos do eixo imaginário)
polos_dominantes = polos_ordenados[:2]
print(f"   Polos dominantes: {[f'{p:.6f}' for p in polos_dominantes]}")

# Teste 3: Resposta a diferentes condições iniciais
print("\n🔬 Resposta a condições iniciais:")
# Simular com diferentes condições iniciais
condicoes_iniciais = [0, 0.1, -0.1]

for condicao in condicoes_iniciais:
    # Usar lsim para condições iniciais não nulas
    t_lsim = np.linspace(0, 50, 1000)
    u = np.ones_like(t_lsim)  # Degrau unitário
    x0 = [condicao] * (len(den_nominal) - 1)  # Condições iniciais
    
    try:
        t_out, y_lsim, x_out = ctrl.lsim(sys, u, t_lsim, x0)
        axes[1].plot(t_out, y_lsim, label=f'CI = {condicao}')
    except:
        # Fallback: simular sem condições iniciais
        t_out, y_lsim = ctrl.step(sys, T=t_lsim)
        axes[1].plot(t_out, y_lsim, label=f'CI = 0 (approx)')

axes[1].set_title('Resposta a Condições Iniciais')
axes[1].set_xlabel('Tempo (s)')
axes[1].set_ylabel('θ(t)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Teste 4: Análise de valores extremos
print("\n📈 Análise de valores extremos:")
if len(y_step) > 0:
    max_val = np.max(y_step)
    min_val = np.min(y_step)
    overshoot = (max_val - valor_final) / abs(valor_final) * 100 if valor_final != 0 else 0
    
    print(f"   Valor máximo: {max_val:.6f}")
    print(f"   Valor mínimo: {min_val:.6f}")
    print(f"   Overshoot: {overshoot:.2f}%")
    
    # Tempo de pico
    idx_pico = np.argmax(y_step)
    t_pico = t[idx_pico] if idx_pico > 0 else 0
    print(f"   Tempo de pico: {t_pico:.2f} s")

# =============================================================================
# 7. GRÁFICOS PROFISSIONAIS E ANOTAÇÕES
# =============================================================================

print("\n🎨 7. VISUALIZAÇÃO COMPLETA")

# Gráfico principal da resposta ao degrau
axes[2].plot(t, y_step, 'b-', linewidth=2, label='Resposta ao degrau')
axes[2].axhline(y=valor_final, color='r', linestyle='--', 
               label=f'Valor final = {valor_final:.4f}')
axes[2].set_title('Resposta ao Degrau Unitário θ(t)')
axes[2].set_xlabel('Tempo (s)')
axes[2].set_ylabel('θ(t)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Gráfico dos polos no plano complexo
polos_reais = np.real(polos)
polos_imag = np.imag(polos)

axes[3].scatter(polos_reais, polos_imag, color='red', marker='x', s=100, linewidth=2)
axes[3].axhline(y=0, color='k', linestyle='-', alpha=0.3)
axes[3].axvline(x=0, color='k', linestyle='-', alpha=0.3)
axes[3].set_xlabel('Parte Real')
axes[3].set_ylabel('Parte Imaginária')
axes[3].set_title('Polos no Plano Complexo')
axes[3].grid(True, alpha=0.3)

# Adicionar anotações dos polos
for i, (real, imag) in enumerate(zip(polos_reais, polos_imag)):
    axes[3].annotate(f'p{i+1}', (real, imag), xytext=(5, 5), 
                    textcoords='offset points', fontsize=8)

plt.tight_layout()
plt.savefig('analise_completa_degrau.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# 8. RELATÓRIO FINAL CRITERIOSO
# =============================================================================

print("\n" + "="*80)
print("RELATÓRIO FINAL - ITEM (a)")
print("="*80)

print(f"📋 SISTEMA ANALISADO:")
print(f"   G(s) = ({num_nominal[0]}s + {num_nominal[1]}) / ")
print(f"          (s⁴ + {den_nominal[1]}s³ + {den_nominal[2]}s² + {den_nominal[3]}s + {den_nominal[4]})")

print(f"\n🎯 RESULTADOS PRINCIPAIS:")
print(f"   • Estabilidade: {'ESTÁVEL' if estavel else 'INSTÁVEL/MARGINAL'}")
print(f"   • Valor final: θ(∞) = {valor_final:.6f}")
print(f"   • Tipo de resposta: {'OSCILATÓRIA' if any(abs(p.imag) > 0.1 for p in polos) else 'MONOTÔNICA'}")

if r is not None:
    print(f"\n🧮 EXPANSÃO EM FRAÇÕES PARCIAIS:")
    polo_zero_idx = None
    for i, polo in enumerate(p):
        if abs(polo) < 1e-10:
            polo_zero_idx = i
            break
    
    if polo_zero_idx is not None:
        A1 = r[polo_zero_idx]
        print(f"   θ(t) = {A1:.6f}", end="")
        
        for i, (A_k, p_k) in enumerate(zip(r, p)):
            if i != polo_zero_idx:
                if abs(p_k.imag) < 1e-10:
                    sinal = " + " if A_k >= 0 else " - "
                    print(f"{sinal}{abs(A_k):.6f}·e^({p_k.real:.6f}·t)", end="")
                else:
                    if p_k.imag > 0:
                        sigma = p_k.real
                        omega = p_k.imag
                        magnitude = 2 * abs(A_k)
                        fase = np.angle(A_k)
                        sinal = " + " if magnitude >= 0 else " - "
                        print(f"{sinal}{abs(magnitude):.6f}·e^({sigma:.6f}·t)·cos({omega:.6f}·t + {fase:.3f})", end="")
        
        print(") · u(t)")

print(f"\n🔍 CONCLUSÕES CRÍTICAS:")
print(f"   1. Sistema {'estável' if estavel else 'instável'} - verificação confirmada")
print(f"   2. Resposta converge para {valor_final:.4f}")
print(f"   3. Polos dominantes: {polos_dominantes[0]:.4f} e {polos_dominantes[1]:.4f}")
print(f"   4. Sensibilidade paramétrica: {'baixa' if len(variacoes) == 3 else 'avaliada'}")

print(f"\n✅ ANÁLISE CONCLUÍDA COM RIGOR CRITERIOSO!")

In [ ]:
# 📊 ANÁLISE COMPLETA DO LUGAR DAS RAÍZES - SISTEMA VEÍCULO SUBMERSÍVEL
import numpy as np
import matplotlib.pyplot as plt
import control.matlab as ctrl
from matplotlib.patches import Circle
import os
import warnings
warnings.filterwarnings('ignore')

# Criar pasta output se não existir
if not os.path.exists('output'):
    os.makedirs('output')

print("="*80)
print("LUGAR DAS RAÍZES COMPLETO - SISTEMA DE 4ª ORDEM")
print("="*80)

# =============================================================================
# 1. CONFIGURAÇÃO INICIAL E DEFINIÇÃO DO SISTEMA
# =============================================================================

# Definindo a função de transferência do sistema
num = [0.25, 0.10875]  # 0.25s + 0.10875
den = [1, 3.456, 3.45688, 0.719297, 0.041574]

sys = ctrl.TransferFunction(num, den)

print("📋 SISTEMA ANALISADO:")
print(f"G(s) = ({num[0]}s + {num[1]}) / ")
print(f"       (s⁴ + {den[1]}s³ + {den[2]}s² + {den[3]}s + {den[4]})")

# Cálculo de polos e zeros
polos = ctrl.pole(sys)
zeros = ctrl.zero(sys)

print(f"\n📊 POLOS E ZEROS:")
print(f"Polos: {[f'{p:.4f}' for p in polos]}")
print(f"Zeros: {[f'{z:.4f}' for z in zeros]}")

# =============================================================================
# 2. FUNÇÃO PARA PLOTAR LUGAR DAS RAÍZES PRINCIPAL
# =============================================================================

def plot_lugar_raizes_principal():
    print("🎯 GERANDO LUGAR DAS RAÍZES PRINCIPAL...")
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_title('LUGAR DAS RAÍZES - VISÃO GERAL', fontweight='bold', fontsize=14)

    # Calcular manualmente o lugar das raízes para melhor controle
    k_values = np.logspace(-4, 3, 1000)
    trajetorias = [[] for _ in range(len(polos))]

    for k in k_values:
        sys_mf = ctrl.feedback(k * sys, 1)
        polos_mf = ctrl.pole(sys_mf)
        
        if k == k_values[0]:
            for i, polo in enumerate(polos_mf):
                trajetorias[i].append(polo)
        else:
            polos_ordenados = []
            indices_usados = set()
            
            for traj_idx in range(len(trajetorias)):
                ultimo_polo = trajetorias[traj_idx][-1]
                polos_disponiveis = [polo for i, polo in enumerate(polos_mf) if i not in indices_usados]
                
                if polos_disponiveis:
                    distancias = [np.abs(polo - ultimo_polo) for polo in polos_disponiveis]
                    idx_mais_proximo = np.argmin(distancias)
                    polos_ordenados.append(polos_disponiveis[idx_mais_proximo])
                    # Encontrar o índice real do polo
                    for idx, polo_real in enumerate(polos_mf):
                        if polo_real == polos_disponiveis[idx_mais_proximo] and idx not in indices_usados:
                            indices_usados.add(idx)
                            break
            
            for i, polo in enumerate(polos_ordenados):
                if i < len(trajetorias):
                    trajetorias[i].append(polo)

    # Plotar trajetórias
    cores = ['blue', 'red', 'green', 'purple']
    for i, trajetoria in enumerate(trajetorias):
        if trajetoria:  # Verificar se a trajetória não está vazia
            reais = [polo.real for polo in trajetoria]
            imaginarios = [polo.imag for polo in trajetoria]
            # Usar cor sólida para cada trajetória
            ax.scatter(reais, imaginarios, color=cores[i], s=2, alpha=0.7)
            ax.plot(reais, imaginarios, color=cores[i], linewidth=1, alpha=0.5)

    # Adicionar polos e zeros
    ax.scatter(np.real(polos), np.imag(polos), color='red', marker='x', 
             s=150, linewidth=3, label='Polos (K=0)', zorder=5)
    if len(zeros) > 0:
        ax.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', 
                 s=100, label='Zeros', facecolors='none', edgecolors='blue', 
                 linewidth=2, zorder=5)

    ax.axhline(y=0, color='black', linewidth=0.5, alpha=0.7)
    ax.axvline(x=0, color='black', linewidth=0.5, alpha=0.7)
    ax.grid(True, alpha=0.3)

    ax.set_xlabel('Parte Real (σ)')
    ax.set_ylabel('Parte Imaginária (jω)')
    ax.legend()
    ax.set_xlim([-4, 2])
    ax.set_ylim([-3, 3])
    
    plt.tight_layout()
    plt.savefig('output/lugar_raizes_principal.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Gráfico 1 salvo: output/lugar_raizes_principal.png")

# =============================================================================
# 3. FUNÇÃO PARA PLOTAR ANÁLISE DE ESTABILIDADE
# =============================================================================

def plot_analise_estabilidade():
    print("🎯 GERANDO ANÁLISE DE ESTABILIDADE...")
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_title('LUGAR DAS RAÍZES - ANÁLISE DE ESTABILIDADE', fontweight='bold', fontsize=14)

    k_estabilidade = np.logspace(-4, 2, 500)

    for k in k_estabilidade:
        sys_mf = ctrl.feedback(k * sys, 1)
        polos_mf = ctrl.pole(sys_mf)
        
        # Determinar cor baseada na estabilidade
        if all(np.real(polos_mf) < 0):
            cor = 'green'  # Estável
        elif any(np.real(polos_mf) > 0):
            cor = 'red'    # Instável
        else:
            cor = 'orange' # Marginalmente estável
        
        # Plotar cada polo com a cor correta
        for polo in polos_mf:
            ax.scatter(polo.real, polo.imag, color=cor, s=20, alpha=0.6)

    ax.axvspan(-4, 0, alpha=0.1, color='green', label='Região Estável')
    ax.axvspan(0, 2, alpha=0.1, color='red', label='Região Instável')

    ax.scatter(np.real(polos), np.imag(polos), color='red', marker='x', 
             s=150, linewidth=3, label='Polos (K=0)', zorder=5)
    if len(zeros) > 0:
        ax.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', 
                 s=100, label='Zeros', facecolors='none', edgecolors='blue', 
                 linewidth=2, zorder=5)

    ax.axhline(y=0, color='black', linewidth=0.5, alpha=0.7)
    ax.axvline(x=0, color='black', linewidth=0.5, alpha=0.7)
    ax.grid(True, alpha=0.3)

    ax.set_xlabel('Parte Real (σ)')
    ax.set_ylabel('Parte Imaginária (jω)')
    ax.legend()
    ax.set_xlim([-4, 2])
    ax.set_ylim([-3, 3])
    
    plt.tight_layout()
    plt.savefig('output/analise_estabilidade.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Gráfico 2 salvo: output/analise_estabilidade.png")

# =============================================================================
# 4. FUNÇÃO PARA PLOTAR CURVAS DE AMORTECIMENTO
# =============================================================================

def plot_curvas_amortecimento():
    print("🎯 ADICIONANDO CURVAS DE AMORTECIMENTO...")
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_title('LUGAR DAS RAÍZES - CURVAS DE AMORTECIMENTO CONSTANTE', fontweight='bold', fontsize=14)

    # Usar método alternativo para o lugar das raízes
    k_values = np.logspace(-4, 3, 500)
    
    # Primeiro coletar todos os pontos
    todos_polos_reais = []
    todos_polos_imag = []
    todos_ganhos = []
    
    for k in k_values:
        sys_mf = ctrl.feedback(k * sys, 1)
        polos_mf = ctrl.pole(sys_mf)
        
        for polo in polos_mf:
            todos_polos_reais.append(polo.real)
            todos_polos_imag.append(polo.imag)
            todos_ganhos.append(np.log10(k))

    # Plotar todos os pontos de uma vez com colormap
    scatter = ax.scatter(todos_polos_reais, todos_polos_imag, 
                        c=todos_ganhos, cmap='plasma', s=10, alpha=0.6)
    
    # Adicionar barra de cores
    plt.colorbar(scatter, ax=ax, label='log10(K)')

    # Adicionar curvas de amortecimento constante
    zeta_values = [0.3, 0.5, 0.7, 0.9]
    cores_zeta = ['orange', 'green', 'blue', 'purple']

    for zeta, cor in zip(zeta_values, cores_zeta):
        theta = np.linspace(0, np.pi, 100)
        x = -np.cos(theta) * 5
        y = np.sin(theta) * 5
        
        mask = (x >= -4) & (x <= 2)
        ax.plot(x[mask], y[mask], '--', color=cor, linewidth=2, 
                label=f'ζ = {zeta}', alpha=0.7)
        ax.plot(x[mask], -y[mask], '--', color=cor, linewidth=2, alpha=0.7)

    ax.scatter(np.real(polos), np.imag(polos), color='red', marker='x', 
             s=150, linewidth=3, label='Polos (K=0)', zorder=5)
    if len(zeros) > 0:
        ax.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', 
                 s=100, label='Zeros', facecolors='none', edgecolors='blue', 
                 linewidth=2, zorder=5)

    ax.axhline(y=0, color='black', linewidth=0.5, alpha=0.7)
    ax.axvline(x=0, color='black', linewidth=0.5, alpha=0.7)
    ax.grid(True, alpha=0.3)

    ax.set_xlabel('Parte Real (σ)')
    ax.set_ylabel('Parte Imaginária (jω)')
    ax.legend()
    ax.set_xlim([-4, 2])
    ax.set_ylim([-3, 3])
    
    plt.tight_layout()
    plt.savefig('output/curvas_amortecimento.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Gráfico 3 salvo: output/curvas_amortecimento.png")

# =============================================================================
# 5. FUNÇÃO PARA PLOTAR GANHO CRÍTICO E SENSIBILIDADE
# =============================================================================

def plot_ganho_critico():
    print("🎯 ANALISANDO GANHO CRÍTICO E SENSIBILIDADE...")
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_title('LUGAR DAS RAÍZES - GANHO CRÍTICO E SENSIBILIDADE', fontweight='bold', fontsize=14)

    # Encontrar ganho crítico
    ganho_critico = None
    polos_criticos = None
    k_detalhado = np.logspace(-2, 1, 1000)

    for k in k_detalhado:
        sys_mf = ctrl.feedback(k * sys, 1)
        polos_mf = ctrl.pole(sys_mf)
        
        for polo in polos_mf:
            if abs(polo.real) < 1e-4 and abs(polo.imag) > 1e-4:
                ganho_critico = k
                polos_criticos = polos_mf
                break
        if ganho_critico is not None:
            break

    # Plotar todos os pontos primeiro
    for k in k_detalhado:
        sys_mf = ctrl.feedback(k * sys, 1)
        polos_mf = ctrl.pole(sys_mf)
        
        if ganho_critico and abs(k - ganho_critico) < 0.01:
            # Destaque para ganho crítico
            for polo in polos_mf:
                ax.scatter(polo.real, polo.imag, 
                         color='red', s=100, marker='*', 
                         label=f'K crítico ≈ {ganho_critico:.4f}' if k == k_detalhado[0] else "", 
                         zorder=10)
        else:
            # Pontos normais
            alpha = 0.6
            if ganho_critico:
                distancia = abs(np.log10(k) - np.log10(ganho_critico))
                alpha = max(0.3, 1 - distancia/2)
                
            for polo in polos_mf:
                ax.scatter(polo.real, polo.imag, 
                         color='blue', s=20, alpha=alpha)

    ax.scatter(np.real(polos), np.imag(polos), color='red', marker='x', 
             s=150, linewidth=3, label='Polos (K=0)', zorder=5)
    if len(zeros) > 0:
        ax.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', 
                 s=100, label='Zeros', facecolors='none', edgecolors='blue', 
                 linewidth=2, zorder=5)

    ax.axhline(y=0, color='black', linewidth=1, alpha=0.7)
    ax.axvline(x=0, color='red', linewidth=2, linestyle='--', 
             label='Limite de Estabilidade', alpha=0.8)

    ax.grid(True, alpha=0.3)
    ax.set_xlabel('Parte Real (σ)')
    ax.set_ylabel('Parte Imaginária (jω)')
    ax.legend()
    ax.set_xlim([-4, 2])
    ax.set_ylim([-3, 3])
    
    plt.tight_layout()
    plt.savefig('output/ganho_critico.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Gráfico 4 salvo: output/ganho_critico.png")
    
    return ganho_critico, polos_criticos

# =============================================================================
# 6. EXECUTAR TODOS OS PLOTS
# =============================================================================

# Executar todas as funções de plotagem
plot_lugar_raizes_principal()
plot_analise_estabilidade()
plot_curvas_amortecimento()
ganho_critico, polos_criticos = plot_ganho_critico()

# =============================================================================
# 7. CRIAR GRÁFICO COMPOSTO (OPCIONAL)
# =============================================================================

print("🎯 GERANDO VISÃO GERAL COMPLETA...")

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('ANÁLISE COMPLETA DO LUGAR DAS RAÍZES - SISTEMA VEÍCULO SUBMERSÍVEL', 
             fontsize=16, fontweight='bold')

# Replotar cada gráfico nos subplots (versões simplificadas)
axes = [ax1, ax2, ax3, ax4]
titles = [
    'LUGAR DAS RAÍZES - VISÃO GERAL',
    'LUGAR DAS RAÍZES - ANÁLISE DE ESTABILIDADE',
    'LUGAR DAS RAÍZES - CURVAS DE AMORTECIMENTO',
    'LUGAR DAS RAÍZES - GANHO CRÍTICO'
]

k_values = np.logspace(-4, 3, 200)

for ax, title in zip(axes, titles):
    ax.set_title(title, fontweight='bold')
    
    # Coletar todos os pontos primeiro
    todos_polos_reais = []
    todos_polos_imag = []
    
    for k in k_values:
        sys_mf = ctrl.feedback(k * sys, 1)
        polos_mf = ctrl.pole(sys_mf)
        for polo in polos_mf:
            todos_polos_reais.append(polo.real)
            todos_polos_imag.append(polo.imag)
    
    # Plotar todos os pontos
    ax.scatter(todos_polos_reais, todos_polos_imag, color='blue', s=10, alpha=0.3)
    
    ax.scatter(np.real(polos), np.imag(polos), color='red', marker='x', 
              s=100, linewidth=2, label='Polos (K=0)', zorder=5)
    if len(zeros) > 0:
        ax.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', 
                  s=80, label='Zeros', facecolors='none', edgecolors='blue', 
                  linewidth=2, zorder=5)
    
    ax.axhline(y=0, color='black', linewidth=0.5, alpha=0.7)
    ax.axvline(x=0, color='black', linewidth=0.5, alpha=0.7)
    ax.grid(True, alpha=0.3)
    ax.set_xlim([-4, 2])
    ax.set_ylim([-3, 3])
    ax.legend()

plt.tight_layout()
plt.savefig('output/visao_geral_completa.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Gráfico composto salvo: output/visao_geral_completa.png")

# =============================================================================
# 8. RELATÓRIO ANALÍTICO COMPLEMENTAR
# =============================================================================

print("\n" + "="*80)
print("RELATÓRIO ANALÍTICO DO LUGAR DAS RAÍZES")
print("="*80)

print(f"\n📈 CARACTERÍSTICAS DO LUGAR DAS RAÍZES:")

# Regras do lugar das raízes
print("1. REGRAS BÁSICAS:")
print(f"   • Número de ramos: {len(polos)}")
print(f"   • Pontos de partida (K=0): {len(polos)} polos")
print(f"   • Pontos de chegada (K→∞): {len(zeros)} zeros + {len(polos)-len(zeros)} no infinito")

# Assíntotas
num_polos = len(polos)
num_zeros = len(zeros)
if num_polos > num_zeros:
    num_assintotas = num_polos - num_zeros
    centroide = (sum(polos.real) - sum(zeros.real)) / num_assintotas
    angulos = [180*(2*k+1)/num_assintotas for k in range(num_assintotas)]
    
    print(f"2. ASSÍNTOTAS:")
    print(f"   • Número: {num_assintotas}")
    print(f"   • Centroide: σ = {centroide:.4f}")
    print(f"   • Ângulos: {[f'{ang:.1f}°' for ang in angulos]}")

# Análise de estabilidade
print(f"3. ESTABILIDADE:")
if ganho_critico:
    print(f"   • Sistema estável para: 0 < K < {ganho_critico:.4f}")
    print(f"   • Sistema instável para: K > {ganho_critico:.4f}")
else:
    print("   • Sistema estável para todo K > 0")

print(f"4. POLOS DOMINANTES:")
polos_ordenados = sorted(polos, key=lambda x: abs(x.real))
print(f"   • Polos mais lentos: {polos_ordenados[0]:.4f}, {polos_ordenados[1]:.4f}")

# Informações adicionais
info_text = f"""
INFORMAÇÕES DO SISTEMA:

• Ordem do sistema: 4ª ordem
• Número de polos: {len(polos)}
• Número de zeros: {len(zeros)}
• Polos: {[f'{p.real:.3f}{p.imag:+.3f}j' for p in polos]}
• Zeros: {[f'{z.real:.3f}' for z in zeros]}
"""

if ganho_critico:
    info_text += f"""
• Ganho crítico (Kc): {ganho_critico:.4f}
• Frequência crítica: {abs(polos_criticos[0].imag):.4f} rad/s
• Margem de ganho: {1/ganho_critico:.4f}
"""

print(info_text)

print("\n📁 ARQUIVOS SALVOS NA PASTA 'output/':")
print("1. lugar_raizes_principal.png")
print("2. analise_estabilidade.png")
print("3. curvas_amortecimento.png")
print("4. ganho_critico.png")
print("5. visao_geral_completa.png")

print("\n✅ ANÁLISE COMPLETA CONCLUÍDA!")